#Obtendo dados do arquivo CSV

In [60]:
import pandas as pd
#define variavel com o local de cada arquivo
arquivo_2020 = '/content/drive/MyDrive/Data Analytics/Projetos/Voos/arquivos/2020 - VRA_20230304002419.csv'
arquivo_2021 = '/content/drive/MyDrive/Data Analytics/Projetos/Voos/arquivos/2021 - VRA_20230304002254.csv'
arquivo_2022 = '/content/drive/MyDrive/Data Analytics/Projetos/Voos/arquivos/2022 - VRA_20230304001948.csv'
arquivo_2023 = '/content/drive/MyDrive/Data Analytics/Projetos/Voos/arquivos/2023 - VRA_20230304002520.csv'

#realiza a leitura dos arquivos csv
dados_2020 = pd.read_csv(arquivo_2020, sep = ';')
dados_2021 = pd.read_csv(arquivo_2021, sep = ';')
dados_2022 = pd.read_csv(arquivo_2022, sep = ';')
dados_2023 = pd.read_csv(arquivo_2023, sep = ';')

#concatena todos os arquivos em um unico dataframe
dados_voos = pd.concat([dados_2020, dados_2021, dados_2022, dados_2023])



/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#Extrai e acrescenta novas informações ao dados

In [61]:
#Filtrando dataframe apenas com as colunas necessárias
#Filtrando apenas dados onde o voo foi realizado
dados_voos = dados_voos.filter(items=\
                     ['Chegada Real','Empresa Aérea', 'Descrição Aeroporto Origem',\
                      'Descrição Aeroporto Destino','Situação Voo'])\
                      .where(dados_voos['Situação Voo'] == 'REALIZADO')\
                      .dropna()

#Nota-se que a coluna ['Descrição Aeroporto Destino'] tem a seguinte estrutura 
#NOME AEROPORTO - CIDADE - ESTADO - PAÍS

#Cria uma nova coluna extraindo apenas os paises de origem do voo
dados_voos['pais_origem'] =  dados_voos['Descrição Aeroporto Origem'].str.split(' - ').str[-1]
#Cria uma nova coluna extraindo apenas os paises de destino do voo
dados_voos['pais_destino'] = dados_voos['Descrição Aeroporto Destino'].str.split(' - ').str[-1]
#Cria uma nova coluna para classificar voos entre nacionais e internacionais
import numpy as np
dados_voos['destino_voo'] = np.where(dados_voos['pais_destino'] == 'BRASIL','Nacional','Internacional' )

#Para voos nacionais, será extraido também cidade e estado do destino.
#Cria uma nova coluna extraindo apenas a cidade de destino do voo
dados_voos['cidade_destino'] = dados_voos['Descrição Aeroporto Destino'].str.split(' - ').str[-3]
#Cria uma nova coluna extraindo apenas o estado de destino do voo
dados_voos['estado_destino'] = dados_voos['Descrição Aeroporto Destino'].str.split(' - ').str[-2]

#Cria coluna mes-ano para agrupamento dos registros
#Converte coluna Chegada Real para formato de data
dados_voos['Chegada Real'] = pd.to_datetime(dados_voos['Chegada Real'])
#Cria coluna com a data em formato mes/ano em formato de string
dados_voos['data'] = dados_voos['Chegada Real'].dt.strftime('%m/%Y')
#Converte a coluna data para formato de data
dados_voos['data'] = pd.to_datetime(dados_voos['data'])


#Filtra o dataframe com os dados necessários

In [62]:
#Cria dataframe de voos nacionais
#Filtra dataframe apenas com dados de voos nacionais partidos do Brasil
dados_voos_nac = dados_voos[dados_voos['destino_voo'] == 'Nacional']
dados_voos_nac = dados_voos_nac[dados_voos_nac['pais_origem'] == 'BRASIL']

#Agrupa o data frame por mes x ano x destino
dados_voos_nac = dados_voos_nac.filter(items = ['data','estado_destino','cidade_destino','Situação Voo'])\
.groupby(['data','estado_destino','cidade_destino'], as_index = False)\
.count()\
.sort_values(by= 'data')

# Exporta dataframe gerado para um arquivo CSV

In [63]:
#Exporta dataframe para arquivo CSV
dados_voos_nac.to_csv('dados_voos_nac.csv', index = False)